In [10]:
# imports
import requests
import pandas as pd
import numpy as np
import os

from pprint import pprint

pd.set_option('display.max_columns', None)
os.chdir('../data')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
# categories = [bars] + [restaurants] + [landmarks] + [convenience store] + [train station]
categories = ([str(i) for i in range(13003, 13026)] + ['13389', '10032']) + [str(i) for i in range(13027, 13390)] + [str(i) for i in range(16000, 16060)] \
    + ['17029'] + ['19066']

In [6]:
def get_venues_fs(latitude, longitude, radius, api_key, categories):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        radius (float): Radius of circle around coordinates to query
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        response: response object from the requests library.
    """
    categories = ",".join(categories)
    url = "https://api.foursquare.com/v3/places/search"
    params = {
  	"ll": str(latitude) + "," + str(longitude),
  	"radius": str(radius),
    "categories": categories
    }
    headers = {
    "Accept": "application/json",
    "Authorization": api_key
    }

    response = requests.get(url, params=params, headers=headers)

    return response

Testing for a single bike station's coordinates.

In [60]:
res = get_venues_fs(35.708596, 139.758269, 1000, FOURSQUARE_API_KEY, categories)

In [61]:
json_dict = res.json()

In [62]:
pprint(json_dict['results'])

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
                           'suffix': '.png'},
                  'id': 13031,
                  'name': 'Burger Joint',
                  'plural_name': 'Burger Joints',
                  'short_name': 'Burgers'},
                 {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
                           'suffix': '.png'},
                  'id': 13145,
                  'name': 'Fast Food Restaurant',
                  'plural_name': 'Fast Food Restaurants',
                  'short_name': 'Fast Food'}],
  'chains': [],
  'closed_bucket': 'LikelyOpen',
  'distance': 116,
  'fsq_id': '4b542534f964a52032b327e3',
  'geocodes': {'main': {'latitude': 35.707565, 'longitude': 139.758191},
               'roof': {'latitude': 35.707565, 'longitude': 139.758191}},
  'link': '/v3/places/4b542534f964a52032b327e3',
  'location': {'address': '本郷4丁目5-10',
               'country': 

In [71]:
station_area_data = pd.json_normalize(json_dict['results'], record_path='categories', meta=['name', 'fsq_id'], record_prefix='categories.')
geo_data = [poi['geocodes']['main'] | {'fsq_id': poi['fsq_id']} for poi in json_dict['results']]
geo_data = pd.json_normalize(geo_data)


In [72]:
geo_data

,latitude,longitude,fsq_id
0,35.707565,139.758191,4b542534f964a52032b327e3
1,35.706507,139.760552,5ceb6cc7588e36002c5dd8e4
2,35.706755,139.761595,5a88e618cbcdee114ecd2f4a
3,35.706614,139.760518,4b5ef33cf964a520519f29e3
4,35.703175,139.755529,4be8c523c5220f47e433aaca
5,35.709307,139.760173,4b690983f964a52096972be3
6,35.704245,139.752004,4b596ca1f964a520e68728e3
7,35.705084,139.748478,4b591810f964a5203d7c28e3
8,35.704032,139.766925,4df843b8d4c02ad734183fff
9,35.711959,139.752359,4b9f1acef964a520ea1337e3


In [73]:
station_area_data

,categories.id,categories.name,categories.short_name,categories.plural_name,categories.icon.prefix,categories.icon.suffix,name,fsq_id
0,13031,Burger Joint,Burgers,Burger Joints,https://ss3.4sqi.net/img/categories_v2/food/bu...,.png,FIRE HOUSE,4b542534f964a52032b327e3
1,13145,Fast Food Restaurant,Fast Food,Fast Food Restaurants,https://ss3.4sqi.net/img/categories_v2/food/fa...,.png,FIRE HOUSE,4b542534f964a52032b327e3
2,13281,Udon Restaurant,Udon,Udon Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ra...,.png,TOKYO LIGHT BLUE HONGO-3,5ceb6cc7588e36002c5dd8e4
3,13272,Ramen Restaurant,Ramen,Ramen Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ra...,.png,Nishino (中華蕎麦 にし乃),5a88e618cbcdee114ecd2f4a
4,13020,Sake Bar,Sake Bar,Sake Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png,加賀屋本郷店,4b5ef33cf964a520519f29e3
5,13280,Tonkatsu Restaurant,Tonkatsu,Tonkatsu Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ja...,.png,Katsukichi (かつ吉),4be8c523c5220f47e433aaca
6,13272,Ramen Restaurant,Ramen,Ramen Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ra...,.png,Yah-Yah-Ya (家家家),4b690983f964a52096972be3
7,10043,Theater,Theater,Theaters,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,Korakuen Hall (後楽園ホール),4b596ca1f964a520e68728e3
8,13263,Japanese Restaurant,Japanese,Japanese Restaurants,https://ss3.4sqi.net/img/categories_v2/food/ja...,.png,Korakuen Hall (後楽園ホール),4b596ca1f964a520e68728e3
9,16017,Garden,Garden,Gardens,https://ss3.4sqi.net/img/categories_v2/parks_o...,.png,Koishikawa Kōrakuen Gardens (小石川後楽園),4b591810f964a5203d7c28e3


In [74]:
station_area_data = pd.merge(station_area_data, geo_data, on='fsq_id').assign(station_name = 'E5-26.本郷4丁目')

In [77]:
station_area_data = station_area_data[['name', 'categories.name', 'latitude', 'longitude', 'station_name']]

In [78]:
station_area_data

,name,categories.name,latitude,longitude,station_name
0,FIRE HOUSE,Burger Joint,35.707565,139.758191,E5-26.本郷4丁目
1,FIRE HOUSE,Fast Food Restaurant,35.707565,139.758191,E5-26.本郷4丁目
2,TOKYO LIGHT BLUE HONGO-3,Udon Restaurant,35.706507,139.760552,E5-26.本郷4丁目
3,Nishino (中華蕎麦 にし乃),Ramen Restaurant,35.706755,139.761595,E5-26.本郷4丁目
4,加賀屋本郷店,Sake Bar,35.706614,139.760518,E5-26.本郷4丁目
5,Katsukichi (かつ吉),Tonkatsu Restaurant,35.703175,139.755529,E5-26.本郷4丁目
6,Yah-Yah-Ya (家家家),Ramen Restaurant,35.709307,139.760173,E5-26.本郷4丁目
7,Korakuen Hall (後楽園ホール),Theater,35.704245,139.752004,E5-26.本郷4丁目
8,Korakuen Hall (後楽園ホール),Japanese Restaurant,35.704245,139.752004,E5-26.本郷4丁目
9,Koishikawa Kōrakuen Gardens (小石川後楽園),Garden,35.705084,139.748478,E5-26.本郷4丁目


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [126]:
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
# categories = [bars] + [restaurants] + [landmarks] + [convenience store] + [train station]
categories = ([str(i) for i in range(13003, 13026)] + ['13389', '10032']) + [str(i) for i in range(13027, 13390)] + [str(i) for i in range(16000, 16060)] \
    + ['17029'] + ['19066']

class FSFramer:
    def __init__(self, api_key, categories, radius=1000):
        self.api_key = api_key
        self.categories = ",".join(categories)
        self.radius = radius
        self.df = pd.DataFrame()


    def get_venues(self, latitude, longitude):
        """
        Get venues from foursquare with a specified place type and coordinates.
        Args:
            latitude (float): latitude for query (must be combined with longitude)
            longitude (float): longitude for query (must be combined with latitude)
            radius (float): Radius of circle around coordinates to query
            api_key (str): foursquare API to use for query
            categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
        
        Returns:
            response: response object from the requests library.
        """
        url = "https://api.foursquare.com/v3/places/search"
        params = {
        "ll": str(latitude) + "," + str(longitude),
        "radius": str(self.radius),
        "categories": self.categories
        }
        headers = {
        "Accept": "application/json",
        "Authorization": self.api_key
        }

        response = requests.get(url, params=params, headers=headers)

        return response
    
    
    def get_station_area_data(self, station, response):
        json_dict = response.json()
        station_area_data = pd.json_normalize(json_dict['results'], record_path='categories', meta=['name', 'fsq_id'], record_prefix='categories.')
        geo_data = [poi['geocodes']['main'] | {'fsq_id': poi['fsq_id']} for poi in json_dict['results']]
        geo_data = pd.json_normalize(geo_data)
        station_area_data = pd.merge(station_area_data, geo_data, on='fsq_id').assign(station_name = station)

        return station_area_data

    def frame(self, station_list, latitude_list, longitude_list, testing = False):
        station_list_length = len(station_list)
        try:
            assert(len(latitude_list) == station_list_length and len(longitude_list) == station_list_length)
        except AssertionError:
            print('longitude_list and latitude_list lengths must be equal to station_list length.')
            return
        else:
            self.df = pd.DataFrame()
            if testing:
                station_list_length = min(station_list_length, 5)
            for i in range(station_list_length):
                response = self.get_venues(latitude_list[i], longitude_list[i])
                station_area_data = self.get_station_area_data(station_list[i], response)
                self.df = pd.concat([self.df, station_area_data])
            self.df.reset_index(drop=True, inplace=True)
            return self.df

In [ ]:
def get_venues(latitude, longitude):
        """
        Get venues from foursquare with a specified place type and coordinates.
        Args:
            latitude (float): latitude for query (must be combined with longitude)
            longitude (float): longitude for query (must be combined with latitude)
            radius (float): Radius of circle around coordinates to query
            api_key (str): foursquare API to use for query
            categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
        
        Returns:
            response: response object from the requests library.
        """
        url = "https://api.foursquare.com/v3/places/search"
        params = {
        "ll": str(latitude) + "," + str(longitude),
        "radius": str(self.radius),
        "categories": self.categories
        }
        headers = {
        "Accept": "application/json",
        "Authorization": self.api_key
        }

        response = requests.get(url, params=params, headers=headers)

        return response

def get_station_area_data(station, response):
        json_dict = response.json()
        station_area_data = pd.json_normalize(json_dict['results'], record_path='categories', meta=['name', 'fsq_id'], record_prefix='categories.')
        geo_data = [poi['geocodes']['main'] | {'fsq_id': poi['fsq_id']} for poi in json_dict['results']]
        geo_data = pd.json_normalize(geo_data)
        station_area_data = pd.merge(station_area_data, geo_data, on='fsq_id').assign(station_name = station)

for station in station_list:
    response = get_venues(station[1], station[2])
    station_area_data = get_station_area_data(station[0], response)
    self.df = pd.concat([self.df, station_area_data])
    self.df.reset_index(drop=True, inplace=True)


In [11]:
docomo_df = pd.read_csv('./docomo_data_time/docomo_data1707416104.csv', index_col=[0])
docomo_df.head()

,empty_slots,free_bikes,latitude,longitude,name,timestamp,extra.last_updated,extra.slots
0,0,5,35.708596,139.758269,E5-26.本郷4丁目,2024-02-08T18:15:54.452000Z,1707416104,5
1,30,5,35.692599,139.770109,A3-42.内神田3丁目,2024-02-08T18:15:54.462000Z,1707416104,35
2,3,2,35.748736,139.574839,T2-76.東中前の森緑地,2024-02-08T18:15:54.463000Z,1707416104,5
3,19,0,35.693408,139.762809,A2-20.神田スクエア,2024-02-08T18:15:54.464000Z,1707416104,19
4,3,4,35.586993,139.736491,K1-79.Tマークシティホテル東京大森,2024-02-08T18:15:54.476000Z,1707416104,7


In [129]:
docomo_fs_framer = FS_Framer(FOURSQUARE_API_KEY, categories, radius=1000)

In [4]:
docomo_stations, latitudes, longitudes = docomo_df['name'].to_list(), docomo_df['latitude'].to_list(), docomo_df['longitude'].to_list()

In [131]:
docomo_area_df = docomo_fs_framer.frame(docomo_stations, latitudes, longitudes)

In [132]:
docomo_area_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18778 entries, 0 to 18777
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   categories.id           18778 non-null  int64  
 1   categories.name         18778 non-null  object 
 2   categories.short_name   18778 non-null  object 
 3   categories.plural_name  18778 non-null  object 
 4   categories.icon.prefix  18778 non-null  object 
 5   categories.icon.suffix  18778 non-null  object 
 6   name                    18778 non-null  object 
 7   fsq_id                  18778 non-null  object 
 8   latitude                18778 non-null  float64
 9   longitude               18778 non-null  float64
 10  station_name            18778 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 1.6+ MB


Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [7]:
YELP_API_KEY = os.getenv('YELP_API_KEY')
# categories = [bars] + [restaurants] + [landmarks] + [convenience store] + [train station]
categories = ['bars', 'nightlife', 'restaurants', 'landmarks', 'parks', 'convenience', 'trainstations']

class YelpFramer:
    def __init__(self, api_key, categories, radius=1000):
        self.api_key = api_key
        self.categories = ",".join(categories)
        self.radius = radius
        self.df = pd.DataFrame()


    def get_venues(self, latitude, longitude):
        """
        Get venues from Yelp with a specified place type and coordinates.
        Args:
            latitude (float): latitude for query (must be combined with longitude)
            longitude (float): longitude for query (must be combined with latitude)
            radius (float): Radius of circle around coordinates to query
            api_key (str): Yelp API to use for query
            categories (str) : Yelp-recognized place type. If not passed no place_type will be specified. Separate terms with commas
        
        Returns:
            response: response object from the requests library.
        """
        url = "https://api.yelp.com/v3/businesses/search"
        params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": self.radius,
        "categories": self.categories
        }
        headers = {
        "Accept": "application/json",
        "Authorization": 'Bearer ' + self.api_key
        }

        response = requests.get(url, params=params, headers=headers)

        return response
    
    
    def get_station_area_data(self, station, response):
        json_dict = response.json()
        station_area_data = pd.json_normalize(json_dict['businesses'])
        station_area_data['categories'] = station_area_data['categories'].apply(lambda x: x[0]['title'])
        station_area_data = station_area_data.assign(station_name = station)

        return station_area_data

    def frame(self, station_list, latitude_list, longitude_list, testing = False, start = 0, api_limit = 500):
        station_list_length = len(station_list)
        try:
            assert(len(latitude_list) == station_list_length and len(longitude_list) == station_list_length)
        except AssertionError:
            print('longitude_list and latitude_list lengths must be equal to station_list length.')
            return
        else:
            self.df = pd.DataFrame()
            station_list_length = min(station_list_length, api_limit)
            if testing:
                station_list_length = min(station_list_length, 2)
            for i in range(start, start + station_list_length):
                response = self.get_venues(latitude_list[i], longitude_list[i])
                station_area_data = self.get_station_area_data(station_list[i], response)
                self.df = pd.concat([self.df, station_area_data])
            self.df.reset_index(drop=True, inplace=True)
            return self.df

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [12]:
yelp_framer = YelpFramer(YELP_API_KEY, categories)
docomo_stations, latitudes, longitudes = docomo_df['name'].to_list(), docomo_df['latitude'].to_list(), docomo_df['longitude'].to_list()


0

In [13]:
yelp_df = yelp_framer.frame(docomo_stations, latitudes, longitudes, start = 475, api_limit=500)


ConnectionError: HTTPSConnectionPool(host='api.yelp.com', port=443): Max retries exceeded with url: /v3/businesses/search?latitude=35.599329&longitude=139.703343&radius=1000&categories=bars%2Cnightlife%2Crestaurants%2Clandmarks%2Cparks%2Cconvenience%2Ctrainstations (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7efdc09c92d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [14]:
yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9192 entries, 0 to 9191
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                9192 non-null   int64  
 1   id                        9192 non-null   object 
 2   alias                     9192 non-null   object 
 3   name                      9192 non-null   object 
 4   image_url                 8939 non-null   object 
 5   is_closed                 9192 non-null   bool   
 6   url                       9192 non-null   object 
 7   review_count              9192 non-null   int64  
 8   categories                9192 non-null   object 
 9   rating                    9192 non-null   float64
 10  transactions              9192 non-null   object 
 11  price                     6227 non-null   object 
 12  phone                     8800 non-null   float64
 13  display_phone             8800 non-null   object 
 14  distance

In [40]:
yelp_df.to_csv('yelp_data_day_0_474.csv')

[{'id': 'qA-jGVDxQtE0chHoX8b05A',
  'alias': 'ファイヤーハウス-文京区-4',
  'name': 'Fire House',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/vqnwxDBUZQzm6pEKeB6TmA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%A4%E3%83%BC%E3%83%8F%E3%82%A6%E3%82%B9-%E6%96%87%E4%BA%AC%E5%8C%BA-4?adjust_creative=ICRr-3etLl33bIREZ0Sxqw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ICRr-3etLl33bIREZ0Sxqw',
  'review_count': 29,
  'categories': [{'alias': 'burgers', 'title': 'Burgers'},
   {'alias': 'cafes', 'title': 'Cafes'}],
  'rating': 4.3,
  'coordinates': {'latitude': 35.7075901, 'longitude': 139.7581639},
  'transactions': [],
  'price': '￥',
  'location': {'address1': '本郷4-5-10',
   'address2': '',
   'address3': '',
   'city': 'Bunkyo',
   'zip_code': '113-0033',
   'country': 'JP',
   'state': '13',
   'display_address': ['本郷4-5-10', 'Bunkyo, 東京都 〒113-0033', 'Japan']},
  'phone': '+81338156044',
  'display_phone': '+81 

{'businesses': [{'id': 'qA-jGVDxQtE0chHoX8b05A',
   'alias': 'ファイヤーハウス-文京区-4',
   'name': 'Fire House',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/vqnwxDBUZQzm6pEKeB6TmA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%A4%E3%83%BC%E3%83%8F%E3%82%A6%E3%82%B9-%E6%96%87%E4%BA%AC%E5%8C%BA-4?adjust_creative=ICRr-3etLl33bIREZ0Sxqw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ICRr-3etLl33bIREZ0Sxqw',
   'review_count': 29,
   'categories': [{'alias': 'burgers', 'title': 'Burgers'},
    {'alias': 'cafes', 'title': 'Cafes'}],
   'rating': 4.3,
   'coordinates': {'latitude': 35.7075901, 'longitude': 139.7581639},
   'transactions': [],
   'price': '￥',
   'location': {'address1': '本郷4-5-10',
    'address2': '',
    'address3': '',
    'city': 'Bunkyo',
    'zip_code': '113-0033',
    'country': 'JP',
    'state': '13',
    'display_address': ['本郷4-5-10', 'Bunkyo, 東京都 〒113-0033', 'Japan']},
   'phone': '+81

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp Fusion API returned a great deal more data per business. However, much of it was not useful for this analysis. In addition, Yelp describes the radius given as a "suggestion" and decreases its size in densely-populated areas, which is a problem when the area that I am searching is Tokyo. In addition, Yelp is business-focused, and states that it will not return "businesses without reviews". While in practice many places that aren't strictly businesses still have Yelp ratings, this does introduce concerns about bias towards places that provide a service. Overall, the Foursquare Places API seemed to provide more trustworthy data for this particular analysis.

Get the top 10 restaurants according to their rating.

I chose to get the top 10 restaurants with more than 100 reviews. Many restaurants were in range of more than 1 station so the duplicates needed to be dropped.

In [4]:
yelp_df = pd.read_csv('yelp_data_day_0_474.csv')

yelp_df[yelp_df['review_count'] >= 100].drop_duplicates('name').nlargest(10, 'rating')

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,station_name
879,879,rUcqLSIYP_mT4aLn9pPExA,牛かつ-もと村-東京駅八重洲口店-中央区-2,Gyukatsu MOTOMURA Tokyo Eki Yaesuguchi,https://s3-media3.fl.yelpcdn.com/bphoto/QQDRKi...,False,https://www.yelp.com/biz/%E7%89%9B%E3%81%8B%E3...,129,Japanese,4.9,[],￥￥,8.133231e+10,+81 3-3231-0337,910.934896,35.681375,139.770603,八重洲1-6-14,NaN,NaN,Chūō,103-0028,JP,13,"['八重洲1-6-14', 'Chūō, 東京都 〒103-0028', 'Japan']",B1-06.伊場仙ビル
329,329,zdA3W7oWPbF4LsNfD35yIw,白秋-渋谷区,Hakushu,https://s3-media3.fl.yelpcdn.com/bphoto/OxaxCO...,False,https://www.yelp.com/biz/%E7%99%BD%E7%A7%8B-%E...,107,Steakhouses,4.8,[],￥￥￥,8.133461e+10,+81 3-3461-0546,787.516545,35.656282,139.700982,桜丘町17-10,MCD Building 1F,NaN,Shibuya,150-0031,JP,13,"['桜丘町17-10', 'MCD Building 1F', 'Shibuya, 東京都 ...",J1-56.青葉台四丁目街かど公園
2931,2931,Cc4IroqPGQFgSQQsgmTQhg,牛かつ-もと村-渋谷店-渋谷区-2,Gyūkatsu Motomura Shibuya,https://s3-media1.fl.yelpcdn.com/bphoto/kVpG1w...,False,https://www.yelp.com/biz/%E7%89%9B%E3%81%8B%E3...,132,Tempura,4.8,[],￥￥,8.133797e+10,+81 3-3797-3735,1198.660567,35.656943,139.703970,渋谷3-18-10,大野ビル2号館 B1F,NaN,Shibuya,979-1308,JP,13,"['渋谷3-18-10', '大野ビル2号館 B1F', 'Shibuya, 東京都 〒97...",M2-13.恵比寿プライムスクエアタワー南
20,20,0xDpum5PoOiI4uDm-uNycA,カラシビ味噌らー麺-鬼金棒-千代田区-2,Karashibi Miso Ramen Kikanbo,https://s3-media1.fl.yelpcdn.com/bphoto/QjaOBT...,False,https://www.yelp.com/biz/%E3%82%AB%E3%83%A9%E3...,131,Ramen,4.7,[],￥,8.133256e+10,+81 3-3256-2960,228.209494,35.693653,139.772415,鍛冶町2-10-10,NaN,NaN,Chiyoda,101-0044,JP,13,"['鍛冶町2-10-10', 'Chiyoda, 東京都 〒101-0044', 'Japan']",A3-42.内神田3丁目
660,660,6gRYBsYGjPPACmcc5OGuQw,牛かつもと村-新宿南口店-新宿区,Gyūkatsu Motomura Shinjuku Minamiguchi,https://s3-media2.fl.yelpcdn.com/bphoto/-H0R_D...,False,https://www.yelp.com/biz/%E7%89%9B%E3%81%8B%E3...,117,Tonkatsu,4.7,[],￥￥,8.133354e+10,+81 3-3354-0171,1089.806825,35.693220,139.701468,新宿3-32-2,モトビル地下1F,NaN,Shinjuku,160-0022,JP,13,"['新宿3-32-2', 'モトビル地下1F', 'Shinjuku, 東京都 〒160-0...",D6-27.新宿三井ビルディング二号館
668,668,B-mK2UEmHN9TRDDtzvMKrg,魚がし日本一-新宿西口店-新宿区,Uogashi Nihon Ichi Shinjuku Nishiguchi,https://s3-media3.fl.yelpcdn.com/bphoto/IHxsk7...,False,https://www.yelp.com/biz/%E9%AD%9A%E3%81%8C%E3...,133,Sushi Bars,4.7,[],￥,8.133349e+10,+81 3-3349-1739,567.436769,35.689970,139.697290,西新宿1-12,河西ビル1F,NaN,Shinjuku,160-0023,JP,13,"['西新宿1-12', '河西ビル1F', 'Shinjuku, 東京都 〒160-0023...",D6-27.新宿三井ビルディング二号館
880,880,Gl2e6kUMnNku2efxtyof1w,ts-たんたん-東京駅京葉ストリート店-千代田区-2,T's Tantan Tokyo Station Keiyo Street,https://s3-media2.fl.yelpcdn.com/bphoto/tvwcYc...,False,https://www.yelp.com/biz/ts-%E3%81%9F%E3%82%93...,187,Vegan,4.7,[],￥,8.133219e+10,+81 3-3218-8040,1270.954862,35.679754,139.766920,丸の内1-9-1,改札内 京葉ストリート,NaN,Chiyoda,100-0005,JP,13,"['丸の内1-9-1', '改札内 京葉ストリート', 'Chiyoda, 東京都 〒100...",B1-06.伊場仙ビル
5797,5797,hB3aFmf7Hqz2ITAoItopiw,新宿御苑-新宿区,Shinjuku Gyoen National Garden,https://s3-media2.fl.yelpcdn.com/bphoto/uWAckY...,False,https://www.yelp.com/biz/%E6%96%B0%E5%AE%BF%E5...,171,Parks,4.7,[],NaN,8.133341e+10,+81 3-3341-1461,693.401479,35.685174,139.710049,内藤町11,NaN,NaN,Shinjuku,160-0014,JP,13,"['内藤町11', 'Shinjuku, 東京都 〒160-0014', 'Japan']",D1-09.NTT四谷ビル
7921,7921,0gxqSYCeVWilIOOOjZtSdA,久兵衛-銀座本店-中央区,Kyubey Ginza,https://s3-media1.fl.yelpcdn.com/bphoto/p51PUA...,False,https://www.yelp.com/biz/%E4%B9%85%E5%85%B5%E8...,136,Sushi Bars,4.7,[],￥￥￥￥,8.133572e+10,+81 3-3571-6523,838.184887,35.668510,139.761222,銀座8-7-6,銀座本店,NaN,Chūō,104-0061,JP,13,"['銀座8-7-6', '銀座本店', 'Chūō, 東京都 〒104-0061', 'Ja...",C1-69.塩釜公園
180,180,gBqIOKuyQFCCzGX_QiMF-A,神戸牛511-港区,Kobebeef 511,https://s3-media3.fl.yelpcdn.com/bphoto/xh2kDg...,False,https://www.yelp.com/biz/%E7%A5%9E%E6%88%B8%E7...,101,Steakhouses,4.6,[],￥￥￥￥,8.136685e+10,+81 3-6685-0511,1085.952588,35.674704,139.735364,港区赤坂,4-3-28ディアプラザ赤坂